In [1]:
import numpy as np
import pandas as pd

In [2]:
# Read all the data files
# Need to comeback and fix this for 2003-2004 and 2004-2005 season
data = []
for year in range(2003, 2020):
    file = pd.read_csv(f'SoccerFiles/PLGames{year}.csv', error_bad_lines = False)
    data.append(file)

b'Skipping line 305: expected 57 fields, saw 72\nSkipping line 306: expected 57 fields, saw 72\nSkipping line 307: expected 57 fields, saw 72\nSkipping line 308: expected 57 fields, saw 72\nSkipping line 309: expected 57 fields, saw 72\nSkipping line 310: expected 57 fields, saw 72\nSkipping line 311: expected 57 fields, saw 72\nSkipping line 312: expected 57 fields, saw 72\nSkipping line 313: expected 57 fields, saw 72\nSkipping line 314: expected 57 fields, saw 72\nSkipping line 315: expected 57 fields, saw 72\nSkipping line 316: expected 57 fields, saw 72\nSkipping line 317: expected 57 fields, saw 72\nSkipping line 318: expected 57 fields, saw 72\nSkipping line 319: expected 57 fields, saw 72\nSkipping line 320: expected 57 fields, saw 72\nSkipping line 321: expected 57 fields, saw 72\nSkipping line 322: expected 57 fields, saw 72\nSkipping line 323: expected 57 fields, saw 72\nSkipping line 324: expected 57 fields, saw 72\nSkipping line 325: expected 57 fields, saw 72\nSkipping li

In [3]:
features = {'HomeGoals': 0, 'AwayGoals': 0, 'Games': 0,
            'Wins': 0, 'Losses': 0, 'Draws': 0,
            'HalfTimeHomeGoals': 0, 'HalfTimeAwayGoals': 0,
            'HalfTimeWins' : 0, 'HalfTimeLosses' : 0, 
            'HalfTimeDraws': 0,
            'HalfTimeChoke': 0, # They were winning at halftime but lost the game
            'HalfTimeClutch': 0, # They were losing at halftime but won the game
            # 'AVGCrowd': 0, 
            'AVGHomeShots': 0, 'AVGAwayShots': 0,
            'AVGHomeShotsTarget': 0, 'AVGAwayShotsTarget': 0,
            #'AVGHomeWoodwork': 0, 'AVGAwayWoodwork': 0, 
            'AVGHomeCorners': 0, 'AVGAwayCorners': 0,
            'AVGHomeFouls': 0, 'AVGAwayFouls': 0,
            #'AVGHomeFreeKicks': 0, 'AVGAwayFreeKicks': 0,
            #'AVGHomeOffsides': 0, 'AVGAwayOffsides': 0,
            'AVGHomeYellow': 0, 'AVGAwayYellow': 0,
            'AVGHomeRed': 0, 'AVGAwayRed': 0,
            'WonAtLeast2': 0, 'WonAtLeast3': 0
           }

In [4]:
AVGFeatures = ['AVGHomeShots', 'AVGAwayShots',
               'AVGHomeShotsTarget', 'AVGAwayShotsTarget',
               #'AVGHomeWoodwork', 'AVGAwayWoodwork', 
               'AVGHomeCorners', 'AVGAwayCorners',
               'AVGHomeFouls', 'AVGAwayFouls',
               #'AVGHomeFreeKicks', 'AVGAwayFreeKicks',
               #'AVGHomeOffsides', 'AVGAwayOffsides',
               'AVGHomeYellow', 'AVGAwayYellow',
               'AVGHomeRed', 'AVGAwayRed']

In [5]:
def HomeTeamStats(Season:{'team':{'str': int}}, row):
    # Add Home Team Stats
    Season[row['HomeTeam']]['HomeGoals'] += row['FTHG'] + 0
    Season[row['HomeTeam']]['HalfTimeHomeGoals'] += row['HTHG']
    Season[row['HomeTeam']]['AVGHomeShots'] += row['HS']
    Season[row['HomeTeam']]['AVGHomeShotsTarget'] += row['HST']
    #Season[row['HomeTeam']]['AVGHomeWoodwork'] += row['HHW']
    Season[row['HomeTeam']]['AVGHomeCorners'] += row['HC']
    Season[row['HomeTeam']]['AVGHomeFouls'] += row['HF']
    #Season[row['HomeTeam']]['AVGHomeFreeKicks'] += row['HFKC']
    #Season[row['HomeTeam']]['AVGHomeOffsides'] += row['HO']
    Season[row['HomeTeam']]['AVGHomeYellow'] += row['HY']
    Season[row['HomeTeam']]['AVGHomeRed'] += row['HR']

In [6]:
def AwayTeamStats(Season: {'team':{'str': int}}, row):
    # Add Away Team Stats
    Season[row['AwayTeam']]['AwayGoals'] += row['FTAG']
    Season[row['AwayTeam']]['HalfTimeAwayGoals'] += row['HTAG']
    Season[row['AwayTeam']]['AVGAwayShots'] += row['AS']
    Season[row['AwayTeam']]['AVGAwayShotsTarget'] += row['AST']
    #Season[row['AwayTeam']]['AVGAwayWoodwork'] += row['AHW']
    Season[row['AwayTeam']]['AVGAwayCorners'] += row['AC']
    Season[row['AwayTeam']]['AVGAwayFouls'] += row['AF']
    #Season[row['AwayTeam']]['AVGAwayFreeKicks'] += row['AFKC']
    #Season[row['AwayTeam']]['AVGAwayOffsides'] += row['AO']
    Season[row['AwayTeam']]['AVGAwayYellow'] += row['AY']
    Season[row['AwayTeam']]['AVGAwayRed'] += row['AR']

In [7]:
def GameWinner(Season: {'team':{'str': int}}, row):
    # Determine Game Winner, Loser, and if there was a draw
    # Also see if a team won by at least 2 goals
    Season[row['HomeTeam']]['Games'] += 1
    Season[row['AwayTeam']]['Games'] += 1
    if row['FTR'] == 'H':
        Season[row['HomeTeam']]['Wins'] += 1
        Season[row['AwayTeam']]['Losses'] += 1
        if (row['FTHG'] - row['FTAG']) >= 2:
            Season[row['HomeTeam']]['WonAtLeast2'] += 1
        if (row['FTHG'] - row['FTAG']) >= 3:
            Season[row['HomeTeam']]['WonAtLeast3'] += 1
            
    elif row['FTR'] == 'A':
        Season[row['HomeTeam']]['Losses'] += 1
        Season[row['AwayTeam']]['Wins'] += 1
        if (row['FTAG'] - row['FTHG']) >= 2:
            Season[row['AwayTeam']]['WonAtLeast2'] += 1
        if (row['FTAG'] - row['FTHG']) >= 3:
            Season[row['AwayTeam']]['WonAtLeast3'] += 1
    else:
        # Tie
        Season[row['HomeTeam']]['Draws'] += 1
        Season[row['AwayTeam']]['Draws'] += 1

In [8]:
def HalfTimeWinner(Season: {'team':{'str': int}}, row):
    # Determine HalfTime Winner
    if row['HTR'] == 'H':
        Season[row['HomeTeam']]['HalfTimeWins'] += 1
        Season[row['AwayTeam']]['HalfTimeLosses'] += 1
    elif row['HTR'] == 'A':
        Season[row['HomeTeam']]['HalfTimeLosses'] += 1
        Season[row['AwayTeam']]['HalfTimeWins'] += 1
    else:
        # Draw
        Season[row['HomeTeam']]['HalfTimeDraws'] += 1
        Season[row['AwayTeam']]['HalfTimeDraws'] += 1

In [9]:
def HalfTimeChoke(Season: {'team':{'str': int}}, row):
    # Was the time winning by halftime and then lost the game
    # or did the reverse happen?
    if (row['HTR'] == 'H') and (row['FTR'] == 'A'):
        Season[row['HomeTeam']]['HalfTimeChoke'] += 1
        Season[row['HomeTeam']]['HalfTimeClutch'] += 1
    elif (row['HTR'] == 'A') and (row['FTR'] == 'H'):
        Season[row['HomeTeam']]['HalfTimeClutch'] += 1
        Season[row['HomeTeam']]['HalfTimeChoke'] += 1

In [10]:
def findAverage(Season: {'team':{'str': int}}, AVGFeatures: ['str']):
    # Find the average for each feature
    for AVGf in AVGFeatures:
        for team, v in Season.items():
            Season[team][AVGf] /= Season[team]['Games']

In [11]:
def appendDataFrame(Season: {'team':{'str': int}}, AllSeasonsdf, year: int):
    for team, v in Season.items():
        v['Season'] = f'{year}-{year + 1}'
        v['Team'] = team
        AllSeasonsdf = AllSeasonsdf.append(v, ignore_index = True)
    return AllSeasonsdf

In [12]:
AllSeasonsdf = pd.DataFrame()
year = 2003
for dfSeason in data:
    Season = dict()
    for index, row in dfSeason.iterrows():
        # New Team for that season
        if row['HomeTeam'] not in Season:
            Season[row['HomeTeam']] = dict(features)
        if row['AwayTeam'] not in Season:
            Season[row['AwayTeam']] = dict(features)
        
        HomeTeamStats(Season, row)
        AwayTeamStats(Season, row)
        GameWinner(Season, row)
        HalfTimeWinner(Season, row)
        HalfTimeChoke(Season, row)
    
    findAverage(Season, AVGFeatures)
    AllSeasonsdf = appendDataFrame(Season, AllSeasonsdf, year)
    year += 1

In [13]:
print(AllSeasonsdf[:4])

   AVGAwayCorners  AVGAwayFouls  AVGAwayRed  AVGAwayShots  AVGAwayShotsTarget  \
0        2.212121      7.030303    0.060606      5.848485            3.393939   
1        2.882353      6.058824    0.058824      4.500000            2.058824   
2        1.735294      6.529412    0.029412      3.558824            1.735294   
3        2.515152      8.030303    0.000000      4.272727            2.212121   

   AVGAwayYellow  AVGHomeCorners  AVGHomeFouls  AVGHomeRed  AVGHomeShots  ...  \
0       0.757576        3.212121      5.909091    0.030303      6.545455  ...   
1       0.705882        3.647059      6.264706    0.029412      7.323529  ...   
2       0.558824        3.264706      7.058824    0.088235      5.117647  ...   
3       1.000000        2.848485      7.060606    0.000000      6.757576  ...   

   HalfTimeHomeGoals  HalfTimeLosses  HalfTimeWins  HomeGoals  Losses  \
0               15.0             3.0          16.0       29.0     0.0   
1                9.0            14.0      

In [14]:
AllSeasonsdf = AllSeasonsdf[['Season', 'Team', 
                             'Wins', 'Losses', 'Draws', 'Games',
                             'HomeGoals', 'AwayGoals',
                             'WonAtLeast2', 'WonAtLeast3',
                             'HalfTimeWins', 'HalfTimeLosses', 'HalfTimeDraws',
                             'HalfTimeHomeGoals', 'HalfTimeAwayGoals', 
                             'HalfTimeClutch', 'HalfTimeChoke',
                             'AVGHomeShots', 'AVGAwayShots',
                             'AVGHomeShotsTarget', 'AVGAwayShotsTarget',
                             'AVGHomeYellow', 'AVGAwayYellow',
                             'AVGHomeRed', 'AVGAwayRed', 
                             'AVGHomeFouls', 'AVGAwayFouls',
                             'AVGHomeCorners', 'AVGAwayCorners']]

In [15]:
AllSeasonsdf.to_csv(f'SoccerFiles/CombinedTeamData.csv', index=False)